In [1]:
# ✅ Install compatible versions
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu125
!pip install -q -U google-genai plyfile


Looking in indexes: https://download.pytorch.org/whl/cu125
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64" (from torch) (from versions: none)
ERROR: No matching distribution found for nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64"


In [2]:
%cd /content/gaussian-splatting/submodules/simple-knn
!python setup.py build_ext --inplace

/content/gaussian-splatting/submodules/simple-knn
running build_ext
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:448: UserWarning: The detected CUDA version (12.5) has a minor version mismatch with the version that was used to compile PyTorch (12.4). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:458: UserWarning: There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 12.5
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
building 'simple_knn._C' extension
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Emitting ninja build file 

In [7]:
# 🔧 One-Click Gaussian Splatting Pipeline (Optimized for A100 GPU with Qt offscreen)
import ipywidgets as widgets
from IPython.display import display, clear_output
import os, re

button = widgets.Button(description="🚀 Run Full Gaussian Splatting Pipeline (GPU Enabled)")
output = widgets.Output()

skip_upload = widgets.Checkbox(value=False, description="Skip Image Upload")
display(skip_upload)

def deduplicate_images(image_dir):
    print("🧹 Deduplicating images by base name...")
    seen = set()
    for img in sorted(os.listdir(image_dir)):
        base = re.sub(r' \(\d+\)| \d+|\(\d+\)', '', img)  # remove duplicates markers like (1), (2)
        if base in seen:
            os.remove(os.path.join(image_dir, img))
            print(f"🗑️ Removed duplicate: {img}")
        else:
            seen.add(base)

def run_pipeline(b):
    with output:
        clear_output()
        print("⚙️ Starting full setup optimized for A100 GPU...")

        # Confirm GPU is active
        import torch
        if not torch.cuda.is_available():
            raise EnvironmentError("❌ GPU not enabled. Go to Runtime > Change runtime type > select 'GPU'.")
        print("🟢 GPU available:", torch.cuda.get_device_name(0))

        # Check CUDA and PyTorch versions
        print("🧠 Torch version:", torch.__version__)
        print("🧠 Torch CUDA version:", torch.version.cuda)
        !nvcc --version

        # 📦 Environment Setup
        print("📦 Installing dependencies...")
        !apt-get update -qq && apt-get install -y cmake libgl1-mesa-dev libglu1-mesa-dev ninja-build build-essential colmap
        !pip install -q -U google-genai plyfile

        %cd /content
        if not os.path.exists("gaussian-splatting"):
            !git clone --recursive https://github.com/camenduru/gaussian-splatting
        else:
            print("📁 Repo already exists.")

        %cd /content/gaussian-splatting/submodules/diff-gaussian-rasterization
        !pip install -e .

        %cd /content/gaussian-splatting/submodules/simple-knn

        # ✅ FIX: Add #include <cfloat> to resolve FLT_MAX error
        !sed -i '1i #include <cfloat>' simple_knn.cu

        # Rebuild with fix
        !python setup.py build_ext --inplace || echo "⚠️ Build failed. Check CUDA/toolchain compatibility."

        %cd /content/gaussian-splatting

        from PIL import Image
        import io, glob, shutil

        if not skip_upload.value:
            # 📁 Upload and Resize Images
            from google.colab import files
            print("📤 Upload your images (they'll be resized to 512x512)...")
            os.makedirs("input_images", exist_ok=True)
            os.makedirs("processed_images", exist_ok=True)
            uploaded = files.upload()

            for filename, data in uploaded.items():
                try:
                    path = os.path.join("processed_images", filename)
                    if os.path.exists(path):
                        print(f"⏭️ Skipping duplicate: {filename}")
                        continue
                    image = Image.open(io.BytesIO(data)).convert("RGB")
                    resized = image.resize((512, 512))
                    resized.save(path)
                    display(resized)
                    print(f"✅ Processed: {path}")
                except Exception as e:
                    print(f"⚠️ Skipped {filename}: {e}")
        else:
            print("⏭️ Skipping upload and resize step.")

        # 🗂️ COLMAP directory setup
        os.makedirs("data/scene/images", exist_ok=True)
        for img_file in glob.glob("processed_images/*"):
            shutil.copy(img_file, "data/scene/images/")

        # 🧹 Deduplicate before COLMAP
        deduplicate_images("data/scene/images")

        print("📁 Copied resized images to: data/scene/images")
        print("📝 Listing images in data/scene/images:")
        !ls data/scene/images

        # 🧠 COLMAP Headless Pipeline (CPU fallback)
        print("🧠 Running COLMAP with CPU (Qt headless mode workaround)...")
        !rm -rf data/scene/sparse/*
        !rm -f data/scene/database.db

        try:
            !colmap feature_extractor \
              --database_path data/scene/database.db \
              --image_path data/scene/images \
              --ImageReader.single_camera 1 \
              --SiftExtraction.use_gpu 0

            !colmap sequential_matcher \
              --database_path data/scene/database.db \
              --SiftMatching.use_gpu 0

            !mkdir -p data/scene/sparse
            !colmap mapper \
              --database_path data/scene/database.db \
              --image_path data/scene/images \
              --output_path data/scene/sparse
        except:
            print("❌ COLMAP encountered an error. Try reloading the notebook and reducing image count or resolution.")
            return

        # 🧾 Convert COLMAP output to transforms.json
        output_dir = "data/scene/sparse/0"
        if not os.path.exists(f"{output_dir}/cameras.txt"):
            print(f"❌ COLMAP failed — no output found at {output_dir}/")
            return

        %cd /content/gaussian-splatting
        !python scripts/colmap2nerf.py

        # 🎯 Train Gaussian Splatting
        print("🚀 Starting Gaussian Splatting training (A100 💥)...")
        !python train.py -s ../data/scene

        print("✅ All done! Gaussian Splatting training finished on A100 🎉")

# Display Button
button.on_click(run_pipeline)
display(button, output)

Checkbox(value=False, description='Skip Image Upload')

Button(description='🚀 Run Full Gaussian Splatting Pipeline (GPU Enabled)', style=ButtonStyle())

Output()

In [4]:
import torch
print(f"🧠 Torch version: {torch.__version__}")
print(f"🧠 Torch CUDA version: {torch.version.cuda}")
!nvcc --version


🧠 Torch version: 2.6.0+cu124
🧠 Torch CUDA version: 12.4
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
